In [1]:
from load_preprocess.functions import Data
from load_preprocess.predictions import Number_Stations
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import config
import json
import numpy as np

from shapely.geometry import LineString, mapping
from itertools import combinations
import re
from scipy import spatial


In [2]:
data = Data(path = 'data/')
df = data.create_df()

df.head(10)

100%|██████████| 8/8 [00:49<00:00,  6.20s/it]


,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line
3,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85932.399845,3076.692877,123934.499714,162432.991960
15,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147881.271737,4015.676440,126130.541927,208410.677869
10,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135138.054662,4994.602982,125508.904876,163440.964713
0,Alsace,0.000106,8.836421e+05,8.322648e+09,171172.687144,4374.466068,102940.396161,188809.381774
17,Picardie,0.000093,1.822331e+06,1.954720e+10,163238.986421,7854.875311,157760.172316,218702.689596
20,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203474.560283,4047.517513,239295.676544,319508.048270
13,Lorraine,0.000085,2.007468e+06,2.366142e+10,189457.934215,4562.427574,173570.336256,235280.099059
6,Bretagne,0.000085,2.333729e+06,2.760417e+10,244626.451176,4037.593151,187474.602114,285248.867037
4,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122829.920651,3688.358818,150571.399619,263736.945386
16,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228887.324618,6852.239326,202901.214557,276408.162313


In [3]:
stations_num = Number_Stations(df_data=df, path_conf='params/config.json', length_to_use='longest_line')
df_f = stations_num.final_station_calculation()

df_f.head(10)

,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line,perc_load,max_length_drive,...,R_2030_total,C_2030,h2_num_2040,R_2040_high_aut,R_2040_mid_aut,R_2040_low_aut,R_2040_total,C_2040,num_stations_2030,num_stations_2040
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,123.934500,162.432992,0.097211,720,...,278.337795,96.0,5832.676123,545.970291,835.013386,289.043095,1670.026771,96.0,3,17
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,126.130542,208.410678,0.066784,720,...,245.344247,96.0,4007.059906,481.252177,736.032741,254.780564,1472.065482,96.0,3,15
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,125.508905,163.440965,0.039934,720,...,115.048781,96.0,2396.023248,225.672609,345.146343,119.473734,690.292687,96.0,1,7
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,102.940396,188.809382,0.029967,720,...,99.734936,96.0,1798.016479,195.633914,299.204809,103.570896,598.409619,96.0,1,6
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,157.760172,218.702690,0.040049,720,...,154.393549,96.0,2402.951193,302.848885,463.180648,160.331763,926.361295,96.0,2,10
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,239.295677,319.508048,0.113242,720,...,637.781537,96.0,6794.540057,1251.033015,1913.344612,662.311596,3826.689224,96.0,7,40
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,173.570336,235.280099,0.030442,720,...,126.250612,96.0,1826.493881,247.645432,378.751837,131.106405,757.503674,96.0,1,8
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,187.474602,285.248867,0.066443,720,...,334.084220,96.0,3986.593524,655.319046,1002.252659,346.933613,2004.505318,96.0,3,21
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,150.571400,263.736945,0.024451,720,...,113.670987,96.0,1467.062383,222.970013,341.012961,118.042948,682.025922,96.0,1,7
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,202.901215,276.408162,0.078611,720,...,383.016338,96.0,4716.679417,751.301279,1149.049015,397.747736,2298.098029,96.0,4,24


Functions (Not needed anymore)

In [ ]:
# def clean_freight_df(df, on_load=True):
    
#     df = df[['TIME', 'TIME.1', '2021']]
    
#     if on_load:
#         freight_type = "number_onload"
#     else:
#         freight_type = "number_offload"
        
#     df.rename(columns={"TIME": "geo_code", "TIME.1": "geo_labels", "2021": freight_type}, inplace=True)
#     df = df.iloc[1:]
#     df_clean = df[df.geo_code.str.startswith("FR").fillna(False)]
#     df_clean.reset_index(inplace=True, drop=True)
    
#     df_clean[freight_type] = df_clean[freight_type].astype('str')
#     df_clean[freight_type] = df_clean[freight_type].str.replace('\.0*$', '', regex=True)
#     df_clean.loc[~(df_clean[freight_type].str.isdigit()), freight_type] = '0'
#     df_clean[freight_type] = df_clean[freight_type].astype('int')
    
#     return df_clean

# def department_region_map(path, df_fr):
#     df_dpts_region = pd.read_csv(path)
#     df_dpts_region.rename(columns={"dep_name":"geo_labels", "region_name":"new_region_name", "old_region_name": "region"}, inplace=True)
#     df_final = pd.merge(df_fr, df_dpts_region[["geo_labels", "region"]], how='left', on=['geo_labels'])
#     df_final = df_final[~(df_final.geo_labels.str.endswith(" "))]
#     return df_final

# def merge_freight(path, df_on, df_off):
    
#     df_onload_fr = clean_freight_df(df_on, on_load=True)
#     df_offload_fr = clean_freight_df(df_off, on_load=False)

#     df_fr = pd.merge(df_onload_fr, df_offload_fr, how='inner', on=['geo_code', 'geo_labels'])
#     df_fr['total_load'] = df_fr.number_offload + df_fr.number_onload 
    
#     df_fr['geo_labels'] = [c[0] for c in df_fr['geo_labels'].str.split("(")]
#     df_fr = department_region_map(path, df_fr)
#     df_fr = df_fr.groupby("region")["total_load"].sum().reset_index()
#     df_fr["full_load"] = df_fr["total_load"].sum()   
#     df_fr["perc_load"] = df_fr["total_load"]/df_fr["full_load"]

#     return df_fr

# def calculate_trucks_stations_peryear(df, year=2030):
    
#     autonomy_high_ms = market_share[0]/sum(market_share)
#     autonomy_medium_ms =market_share[1]/sum(market_share)
#     autonomy_low_ms = market_share[2]/sum(market_share)
#     autonomy_high_km = autonomy_share[0]
#     autonomy_medium_km = autonomy_share[1]
#     autonomy_low_km = autonomy_share[2]
    
#     if (year not in [2030, 2040]):
#         year = 2030
        
#     if year==2030:
#         H2_trucks_num = H2_trucks_2030
#     else:
#         H2_trucks_num = H2_trucks_2040
        
#     df["h2_num_"+str(year)] = H2_trucks_num*df["perc_load"]
#     df["R_"+str(year)+"_high_aut"] = autonomy_high_ms*df["h2_num_"+str(year)]*df["avg_distance_high_aut"]/autonomy_high_km
#     df["R_"+str(year)+"_mid_aut"] = autonomy_medium_ms*df["h2_num_"+str(year)]*df["avg_distance_midlow_aut"]/autonomy_medium_km
#     df["R_"+str(year)+"_low_aut"] = autonomy_low_ms*df["h2_num_"+str(year)]*df["avg_distance_midlow_aut"]/autonomy_low_km
#     df["R_"+str(year)+"_total"] = df["R_"+str(year)+"_high_aut"] + df["R_"+str(year)+"_mid_aut"] + df["R_"+str(year)+"_low_aut"]
#     df["C_"+str(year)] = open_time/avg_time_fill
    
#     return df
    
# def calculate_stations(df, year=2030):
    
#     if (year not in [2030, 2040]):
#         year = 2030
        
#     df["num_stations_"+str(year)] = (df["R_"+str(year)+"_total"] / df["C_"+str(year)]).round().astype(int)

#     return df

# def calculate_number_stations(df, length_to_use='longest_line'):
#     """_summary_

#     Args:
#         df : _description_
#         length_to_use : can be 'longest_line', 'diameter' or 'length_max'.

#     Returns:
#         _type_: _description_
#     """

#     if (length_to_use not in ['longest_line', 'diameter', 'length_max']):
#         length_to_use = 'longest_line'
        
#     df["max_length_drive"] = max_hours_drive*avg_speed_kmh
#     df[["length_max", "length_mean", "diameter", "longest_line"]] = df[["length_max", "length_mean", "diameter", "longest_line"]]/1e3
#     df["avg_distance_high_aut"] = df[["max_length_drive", length_to_use]].min(axis=1) # This would be updated either by diameter or longest point
#     df["avg_distance_midlow_aut"] = 0.6*df["avg_distance_high_aut"]#df_new[["max_length_drive", "length_mean"]].min(axis=1)
    
#     df = calculate_trucks_stations_peryear(df, year=2030)
#     df = calculate_trucks_stations_peryear(df, year=2040)
    
#     df = calculate_stations(df, year=2030)
#     df = calculate_stations(df, year=2040)

#     return df

Execute functions (Not needed)

In [ ]:
# path_conf = "params/config.json"
# conf = json.load(open(path_conf, "r"))
# locals().update(conf)

# df_on = pd.read_excel(path_on_freight, sheet_name='Sheet 1', skiprows=8)
# df_off = pd.read_excel(path_off_freight, sheet_name='Sheet 1', skiprows=8)

# df_fr = merge_freight(path_region_dpt_map, df_on, df_off)
# df_new = pd.merge(df, df_fr[["region", "perc_load"]], how="left", on="region")

# df_new = calculate_number_stations(df_new)

# display(df_fr.head(10))

# df_new.head(10)